#### The objective of the project is to learn how to implement a simple image classification pipeline based on the k-Nearest Neighbour and a deep neural network. The goals of this assignment are as follows:

● Understand the basic Image Classification pipeline and the data-driven approach (train/predict stages)

● Data fetching and understand the train/val/test splits.

● Implement and apply an optimal k-Nearest Neighbor (kNN) classifier (7.5 points)

● Print the classification metric report (2.5 points)

● Implement and apply a deep neural network classifier including (feedforward neural network, RELU activations) (5 points)

● Understand and be able to implement (vectorized) backpropagation (cost stochastic gradient descent, cross entropy loss, cost functions) (2.5 points)

● Implement batch normalization for training the neural network (2.5 points)

● Understand the differences and trade-offs between traditional and NN classifiers with the help of classification metrics (5 points)

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras as ks
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import h5py
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [3]:
img_df = h5py.File('SVHN_single_grey1.h5','r+') 

In [14]:
# List all groups
print("Keys: %s" % img_df.keys())

Keys: <KeysViewHDF5 ['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']>


### Fetching Data

In [15]:
x_text = img_df['X_test']
x_train = img_df['X_train']
x_val = img_df['X_val']
y_test = img_df['y_test']
y_train = img_df['y_train']
y_val = img_df['y_val']

In [17]:
print (x_text.shape)
print (x_train.shape)
print (x_val.shape)
print (y_test.shape)
print (y_train.shape)
print (y_val.shape)

(18000, 32, 32)
(42000, 32, 32)
(60000, 32, 32)
(18000,)
(42000,)
(60000,)


####  Implement and apply an optimal k-Nearest Neighbor (kNN) classifier (7.5 points)

In [18]:
# Using KNN Classifier
model_knn = KNeighborsClassifier(n_neighbors=17)

In [19]:
# flattening the data
X_train = np.reshape(x_train,(42000,32*32))

In [20]:
model_knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=17, p=2,
           weights='uniform')

In [22]:
# Change the dimension from 3D to 2D and predict
X_text = np.reshape(x_text,(18000,32*32))
prediction = model_knn.predict(X_text)

#### Print the classification metric report (2.5 points)

In [24]:
print("Accuracy:", metrics.accuracy_score(prediction,y_test))

Accuracy: 0.5287222222222222


In [25]:
print("Confusion Metrix:   \n", metrics.confusion_matrix(prediction,y_test))

Confusion Metrix:   
 [[1274   99   94  129  105  165  319  100  247  323]
 [  66 1335  226  260  249  172  128  209  119  142]
 [  34   55 1000  135   41   47   40  110   68   71]
 [  35   95   95  736   60  269   67   88  114   89]
 [  46   69   44   47 1177   58  135   27   94   71]
 [  45   36   37  160   16  693  116   37  115   93]
 [  98   31   35   36   54  137  744   37  265   55]
 [  39   46  141   57   19   31   21 1128   25   73]
 [  78   26   52   94   43  119  202   24  654  111]
 [  99   36   79   65   48   77   60   48  111  776]]


In [27]:
cr = metrics.classification_report(y_test,prediction)
print(cr)

              precision    recall  f1-score   support

           0       0.45      0.70      0.55      1814
           1       0.46      0.73      0.56      1828
           2       0.62      0.55      0.59      1803
           3       0.45      0.43      0.44      1719
           4       0.67      0.65      0.66      1812
           5       0.51      0.39      0.44      1768
           6       0.50      0.41      0.45      1832
           7       0.71      0.62      0.67      1808
           8       0.47      0.36      0.41      1812
           9       0.55      0.43      0.48      1804

   micro avg       0.53      0.53      0.53     18000
   macro avg       0.54      0.53      0.52     18000
weighted avg       0.54      0.53      0.52     18000



#### Implement and apply a deep neural network classifier including (feedforward neural network, RELU activations) (5 points)

In [32]:
#Initialize Sequential model
model_nn = ks.models.Sequential()

In [33]:
#Reshape data from 2D to 1D -> 32X32 to 1024
model_nn.add(ks.layers.Reshape((1024,),input_shape=(32,32)))

#### Implement batch normalization for training the neural network

In [34]:
#Normalize the data
model_nn.add(ks.layers.BatchNormalization())

In [35]:
model_nn.add(ks.layers.Dense(150, activation='relu'))
model_nn.add(ks.layers.Dense(100, activation='relu'))

In [36]:
#Output layer
model_nn.add(ks.layers.Dense(10, activation='softmax', name='Output'))

#### Understand and be able to implement (vectorized) backpropagation (cost stochastic gradient descent, cross entropy loss, cost functions) (2.5 points)

In [38]:
model_nn.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
# Change train and test labels into one-hot vectors
trainY = ks.utils.to_categorical(y_train, num_classes=10)
testY = ks.utils.to_categorical(y_test, num_classes=10)

In [40]:
#Train the model
features_val_arr = np.array(x_text)
model_nn.fit(x_train,trainY,          
          validation_data=(features_val_arr, testY),
          epochs=75,
          batch_size=150,validation_split = 0.01,
         shuffle='batch')

Instructions for updating:
Use tf.cast instead.
Train on 42000 samples, validate on 18000 samples
Epoch 1/75
42000/42000 [==============================] - 3s 78us/step - loss: 2.1719 - acc: 0.2299 - val_loss: 1.9331 - val_acc: 0.3748
Epoch 2/75
42000/42000 [==============================] - 2s 59us/step - loss: 1.7271 - acc: 0.4634 - val_loss: 1.5270 - val_acc: 0.5457
Epoch 3/75
42000/42000 [==============================] - 3s 61us/step - loss: 1.4048 - acc: 0.5872 - val_loss: 1.2812 - val_acc: 0.6288
Epoch 4/75
42000/42000 [==============================] - 3s 64us/step - loss: 1.2164 - acc: 0.6481 - val_loss: 1.1366 - val_acc: 0.6743
Epoch 5/75
42000/42000 [==============================] - 3s 64us/step - loss: 1.1000 - acc: 0.6803 - val_loss: 1.0456 - val_acc: 0.6976
Epoch 6/75
42000/42000 [==============================] - 3s 63us/step - loss: 1.0191 - acc: 0.7037 - val_loss: 0.9874 - val_acc: 0.7133
Epoch 7/75
42000/42000 [==============================] - 3s 68us/step - loss: 0

Epoch 58/75
42000/42000 [==============================] - 3s 82us/step - loss: 0.3534 - acc: 0.8952 - val_loss: 0.5894 - val_acc: 0.8388
Epoch 59/75
42000/42000 [==============================] - 4s 84us/step - loss: 0.3469 - acc: 0.8966 - val_loss: 0.5962 - val_acc: 0.8352
Epoch 60/75
42000/42000 [==============================] - 3s 81us/step - loss: 0.3453 - acc: 0.8975 - val_loss: 0.5834 - val_acc: 0.8407
Epoch 61/75
42000/42000 [==============================] - 3s 82us/step - loss: 0.3403 - acc: 0.8981 - val_loss: 0.5816 - val_acc: 0.8413
Epoch 62/75
42000/42000 [==============================] - 3s 81us/step - loss: 0.3369 - acc: 0.9010 - val_loss: 0.5944 - val_acc: 0.8385
Epoch 63/75
42000/42000 [==============================] - 3s 83us/step - loss: 0.3344 - acc: 0.9000 - val_loss: 0.5842 - val_acc: 0.8412
Epoch 64/75
42000/42000 [==============================] - 3s 81us/step - loss: 0.3314 - acc: 0.9016 - val_loss: 0.5818 - val_acc: 0.8417
Epoch 65/75
42000/42000 [=========

In [41]:
# model evaluation
evaluate = model_nn.evaluate(x_text, testY)
print(evaluate)

18000/18000 [==============================] - 1s 66us/step
[0.5984484396908019, 0.8418888888888889]


In [43]:
# predict the model
y_predict=model_nn.predict_classes(x_text)

In [44]:
cr=metrics.classification_report(y_test,y_predict)
print(cr)

              precision    recall  f1-score   support

           0       0.85      0.89      0.87      1814
           1       0.83      0.86      0.85      1828
           2       0.87      0.85      0.86      1803
           3       0.80      0.78      0.79      1719
           4       0.88      0.87      0.88      1812
           5       0.81      0.83      0.82      1768
           6       0.84      0.83      0.83      1832
           7       0.89      0.87      0.88      1808
           8       0.81      0.81      0.81      1812
           9       0.84      0.83      0.83      1804

   micro avg       0.84      0.84      0.84     18000
   macro avg       0.84      0.84      0.84     18000
weighted avg       0.84      0.84      0.84     18000



#### Understand the differences and trade-offs between traditional and NN classifiers with the help of classification metrics (5 points)

 ##### KNN does not perform well compared to DNN
 
KNN: 

Accuracy - 52.87%

Precision - 54%

Recall - 53% 

F1 Score - 52%


DNN:

Accuracy - 84.18%

Precision - 84%

Recall - 84% 

F1 Score - 84%

It is clear from the above that DNN performs better than KNN. Neural network the hidden layers learn the image features by adjusting the weights and is able to classify better than KNN